In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sklearn
from sklearn.linear_model import Lasso, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split

Add appts, last month/year?, treasury yield, home sales?, and more stocks?

In [2]:
data = pd.read_csv("data/out/features.csv")
len(data), len(data.columns)

(50, 53)

In [3]:
X, y = data.drop(columns = ['n']), data['n']

,month,n,oil_lag3,oil_lag6,oil_lag12,ngf2_lag6,bf90_lag3,lhpi_lag3,lhpi_lag6,lhpi_lag12,...,FB_lag3,FB_lag6,FB_lag12,TSLA_lag12,g_GM_lag6,g_car_lag6,g_car_lag12,g_car_sales_near_me_lag3,g_car_sales_near_me_lag6,g_car_sales_near_me_lag12
0,1,352,43.66,50.51,44.83,22,98.01,180.057932,173.862415,165.516015,...,14.81,14.83,14.71,203.60,73,84,73,14,17,10
1,2,330,38.93,47.01,56.83,21,97.76,182.086194,175.938576,166.704482,...,14.33,13.87,16.34,203.34,69,84,76,15,13,13
2,3,323,31.27,43.58,51.06,62,97.78,183.938698,177.945896,168.099462,...,14.09,13.57,16.14,188.77,75,77,78,17,15,11
3,4,369,29.11,43.66,62.21,64,97.81,185.721838,180.057932,169.464578,...,11.94,14.81,15.80,226.05,75,76,80,24,14,9
4,5,392,30.13,38.93,60.47,27,97.74,187.408127,182.086194,170.716319,...,12.51,14.33,15.17,250.80,74,72,83,30,15,10
5,6,329,34.33,31.27,58.79,103,97.79,188.896228,183.938698,172.080363,...,13.50,14.09,15.01,268.26,71,71,86,30,17,15
6,7,420,42.70,29.11,50.51,56,97.91,190.465488,185.721838,173.862415,...,13.56,11.94,14.83,266.15,72,78,84,27,24,17
7,8,359,45.15,30.13,47.01,37,98.01,192.320370,187.408127,175.938576,...,13.49,12.51,13.87,249.06,73,81,84,35,30,13
8,9,368,46.27,34.33,43.58,57,98.12,194.048193,188.896228,177.945896,...,12.57,13.50,13.57,248.40,73,87,77,35,30,15
9,10,367,38.97,42.70,43.66,82,98.17,195.710311,190.465488,180.057932,...,12.66,13.56,14.81,206.93,73,87,76,39,27,14


In [5]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0, random_state=69)

In [26]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', Lasso())
])

In [28]:
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.0, 5, 0.5)},
                      cv = 5,
                      scoring = 'neg_mean_squared_error',
                      verbose = 3)

In [29]:
search.fit(X, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END .............model__alpha=0.0;, score=-19165.095 total time=   0.0s
[CV 2/5] END ..............model__alpha=0.0;, score=-4314.839 total time=   0.0s
[CV 3/5] END .............model__alpha=0.0;, score=-13759.669 total time=   0.0s
[CV 4/5] END ..............model__alpha=0.0;, score=-4351.354 total time=   0.0s
[CV 5/5] END ..............model__alpha=0.0;, score=-6982.496 total time=   0.0s
[CV 1/5] END ..............model__alpha=0.5;, score=-2434.352 total time=   0.0s
[CV 2/5] END ..............model__alpha=0.5;, score=-1700.203 total time=   0.0s
[CV 3/5] END ..............model__alpha=0.5;, score=-4989.044 total time=   0.0s
[CV 4/5] END ..............model__alpha=0.5;, score=-2957.175 total time=   0.0s
[CV 5/5] END ..............model__alpha=0.5;, score=-2643.122 total time=   0.0s
[CV 1/5] END ..............model__alpha=1.0;, score=-3543.539 total time=   0.0s
[CV 2/5] END ..............model__alpha=1.0;, sc

C:\Users\keato\AppData\Roaming\Python\Python37\site-packages\sklearn\pipeline.py:394: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\keato\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
C:\Users\keato\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.914e+02, tolerance: 8.321e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solver

[CV 3/5] END ..............model__alpha=1.5;, score=-4817.778 total time=   0.0s
[CV 4/5] END ..............model__alpha=1.5;, score=-3012.578 total time=   0.0s
[CV 5/5] END ..............model__alpha=1.5;, score=-1401.998 total time=   0.0s
[CV 1/5] END ..............model__alpha=2.0;, score=-2865.903 total time=   0.0s
[CV 2/5] END ..............model__alpha=2.0;, score=-1554.611 total time=   0.0s
[CV 3/5] END ..............model__alpha=2.0;, score=-4500.251 total time=   0.0s
[CV 4/5] END ..............model__alpha=2.0;, score=-3008.483 total time=   0.0s
[CV 5/5] END ..............model__alpha=2.0;, score=-1255.366 total time=   0.0s
[CV 1/5] END ..............model__alpha=2.5;, score=-3137.878 total time=   0.0s
[CV 2/5] END ..............model__alpha=2.5;, score=-1547.316 total time=   0.0s
[CV 3/5] END ..............model__alpha=2.5;, score=-4138.565 total time=   0.0s
[CV 4/5] END ..............model__alpha=2.5;, score=-2999.575 total time=   0.0s
[CV 5/5] END ..............m

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5])},
             scoring='neg_mean_squared_error', verbose=3)

In [30]:
search.best_params_

{'model__alpha': 4.5}

In [31]:
coef = search.best_estimator_[1].coef_

In [32]:
out = X.iloc[:, coef != 0]
out.columns

Index(['oil_lag6', 'oil_lag12', 'ngf2_lag6', 'durable_lag12', 'altsales_lag3',
       'GM_lag12', 'TSLA_lag12'],
      dtype='object')

In [33]:
out.insert(0, 'n', y, True)
out.insert(0, 'month', X['month'], True)

In [34]:
out.to_csv("data/out/lasso.csv")

In [35]:
out

,month,n,oil_lag6,oil_lag12,ngf2_lag6,durable_lag12,altsales_lag3,GM_lag12,TSLA_lag12
0,1,352,50.51,44.83,22,224061,17.818,32.62,203.60
1,2,330,47.01,56.83,21,218966,17.863,37.31,203.34
2,3,323,43.58,51.06,62,227573,17.081,37.50,188.77
3,4,369,43.66,62.21,64,224339,17.610,35.06,226.05
4,5,392,38.93,60.47,27,217332,17.640,35.97,250.80
5,6,329,31.27,58.79,103,232766,16.826,33.33,268.26
6,7,420,29.11,50.51,56,230189,17.247,31.51,266.15
7,8,359,30.13,47.01,37,222647,17.288,29.44,249.06
8,9,368,34.33,43.58,57,218738,17.334,30.02,248.40
9,10,367,42.70,43.66,82,226032,17.744,34.91,206.93
